<a href="https://colab.research.google.com/github/hizmarck/hacking-civico/blob/master/tareas/03_exploracion.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Exploración de datos: COVID-19 en México

Los datos a utilizar son los datos abiertos sobre COVID-19 en México, los cuales pueden ser accedidos directamente desde el portal: https://www.gob.mx/salud/documentos/datos-abiertos-152127


In [1]:
# Para descargar el conjunto de datos
!wget http://datosabiertos.salud.gob.mx/gobmx/salud/datos_abiertos/datos_abiertos_covid19.zip

# Para descargar el diccionario de los datos
!wget http://epidemiologia.salud.gob.mx/gobmx/salud/datos_abiertos/diccionario_datos_covid19.zip

# Descomprimimos ambas carpetas y movemos archivos
!unzip datos_abiertos_covid19.zip
!unzip diccionario_datos_covid19.zip
!mv ./diccionario_datos_covid19/*.xlsx .
!mv *COVID19MEXICO.csv COVID19MEXICO.csv

# Eliminamos las carpetas comprimidas
!rm -rf diccionario_datos_covid19
!rm datos_abiertos_covid19.zip
!rm diccionario_datos_covid19.zip

--2020-09-15 03:01:05--  http://datosabiertos.salud.gob.mx/gobmx/salud/datos_abiertos/datos_abiertos_covid19.zip
Resolving datosabiertos.salud.gob.mx (datosabiertos.salud.gob.mx)... 187.210.186.146
Connecting to datosabiertos.salud.gob.mx (datosabiertos.salud.gob.mx)|187.210.186.146|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 32185209 (31M) [application/x-zip-compressed]
Saving to: ‘datos_abiertos_covid19.zip’

datos_abiertos_covi 100%[===================>]  30.69M  15.6MB/s    in 2.0s    

2020-09-15 03:01:07 (15.6 MB/s) - ‘datos_abiertos_covid19.zip’ saved [32185209/32185209]

--2020-09-15 03:01:07--  http://epidemiologia.salud.gob.mx/gobmx/salud/datos_abiertos/diccionario_datos_covid19.zip
Resolving epidemiologia.salud.gob.mx (epidemiologia.salud.gob.mx)... 187.191.75.207
Connecting to epidemiologia.salud.gob.mx (epidemiologia.salud.gob.mx)|187.191.75.207|:80... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: https://ep

## Carga



In [2]:
import statistics

import pandas as pd
import numpy as np

In [3]:
# Especificamos el nombre del archivo por abrir
nombre_archivo = 'COVID19MEXICO.csv'

# Utilizamos la función de Pandas para cargar un archivo CSV
data = pd.read_csv(nombre_archivo, encoding='latin-1')

## Descriptores.

In [5]:
nombre_diccionario = 'Descriptores_0419.xlsx'
diccionario = pd.read_excel(nombre_diccionario)

## Catálogos.

In [7]:
nombre_cat = 'Catalogos_0412.xlsx'
sheets = ['ORIGEN', 'SECTOR', 'SEXO', 'TIPO_PACIENTE', 'SI_NO', 'NACIONALIDAD', 'RESULTADO', 'de ENTIDADES', 'MUNICIPIOS']
catalogues = {}
for sheet in sheets:
  catalogue = pd.read_excel(nombre_cat, sheet_name='Catálogo ' + sheet)
  catalogue.columns = catalogue.columns.str.replace(' ', '_')
  catalogue.columns = catalogue.columns.str.replace('.', '')
  catalogue.columns = catalogue.columns.str.lower()
  new_catalogue = {sheet: catalogue}
  catalogues.update(new_catalogue)

###Limpieza

In [11]:
# limpieza del nombre de las columnas, remover espacios, carácteres especiales y pasar a minúsculas
data.columns = data.columns.str.replace(' ', '_')
data.columns = data.columns.str.replace('.', '')
data.columns = data.columns.str.lower()

In [12]:
# convertimos a fecha algunas columnas
def cast_datetime_cols(df):
    timestamp_cols = [col for col in df.columns if "fecha" in col]
    df[timestamp_cols] = df[timestamp_cols].apply(lambda date_col: pd.to_datetime(date_col, errors="coerce"), axis=0)
    return df

data = cast_datetime_cols(data)

In [14]:
# unimos los datos con los catalogos para tener mejor entendimiento de ellos
data = (
    # unimos con el catálogo MUNICIPIOS y ENTIDADES
    data.merge(catalogues['de ENTIDADES'], how='left', left_on='entidad_res', right_on='clave_entidad').
    drop(columns=['entidad_res', 'abreviatura']).
    rename(columns={'entidad_federativa':'entidad_res'}).
    merge(catalogues['MUNICIPIOS'], how='left', left_on=['clave_entidad','municipio_res'], right_on=['clave_entidad','clave_municipio']).
    drop(columns=['municipio_res', 'clave_entidad', 'clave_municipio']).
    rename(columns={'municipio':'municipio_res'})
)

In [15]:
# unimos con el catálogo de ENTIDADES
entidades_cols = [col for col in data.columns if "entidad" in col and col != 'entidad_res']
for col in entidades_cols:
    data = (
        data.merge(catalogues['de ENTIDADES'], how='left', left_on=col, right_on='clave_entidad').
        drop(columns=[col, 'abreviatura', 'clave_entidad']).
        rename(columns={'entidad_federativa':col})
    )

In [16]:
# unimos con el catálogo SI_NO
sino_cols = ['intubado', 'neumonia', 'embarazo', 'habla_lengua_indig', 'diabetes', 'epoc', 'asma', 'inmusupr',
             'hipertension', 'otra_com', 'cardiovascular', 'obesidad', 'renal_cronica', 'tabaquismo', 
             'otro_caso', 'migrante', 'uci']
for col in sino_cols:
    data = (
        data.merge(catalogues['SI_NO'], how='left', left_on=col, right_on='clave').
        drop(columns=[col, 'clave']).
        rename(columns={'descripción':col})
    )

> **Nota:**
>
> Si ejecutaste una versión previa del cuaderno, notarás que en la siguiente sección aparecía un error. A continuación dejamos como un pequeño paréntesis un poco más de limpieza sobre el cuaderno.

1. Corregimos el nombre de las columnas:

In [21]:
catalogues['RESULTADO'].columns

Index(['clave', 'descripción'], dtype='object')

In [18]:
catalogues['RESULTADO'].columns = ['clave', 'descripción']
catalogues['RESULTADO'].columns

Index(['clave', 'descripción'], dtype='object')

2. Removemos la (primera) fila que no nos es útil.

In [22]:
catalogues['RESULTADO'] = catalogues['RESULTADO'].drop(index=0)

3. Convertimos el tipo de dato.

In [23]:
catalogues['RESULTADO']['clave'] = catalogues['RESULTADO']['clave'].astype(int)

> Fin del paréntesis. Hasta esta sección, esta parte de limpieza permite continuar con el proceso de manera normal.

In [24]:
# unimos otras columnnas que comparten lógica
other_cols = ['origen', 'sector', 'sexo', 'tipo_paciente', 'nacionalidad', 'resultado']
for col, cat in zip(other_cols, [x.upper() for x in other_cols]):
    data = (
        data.merge(catalogues[cat], how='left', left_on=col, right_on='clave').
        drop(columns=[col, 'clave']).
        rename(columns={'descripción':col})
    )

In [25]:
# creamos algunas variables que podrían ser útiles después
data['defuncion'] = ~data.fecha_def.isna()
data['mes_def'] = data.fecha_def.dt.month
data['año_def'] = data.fecha_def.dt.year
data['mes_ingreso'] = data.fecha_ingreso.dt.month
data['año_ingreso'] = data.fecha_ingreso.dt.year

###Filtros

**Distintas formas de filtrar lo mismo**

In [27]:
# usando query
data.query('nacionalidad == "MEXICANA"').head(2)

,fecha_actualizacion,id_registro,fecha_ingreso,fecha_sintomas,fecha_def,edad,pais_nacionalidad,pais_origen,entidad_res,municipio_res,entidad_um,entidad_nac,intubado,neumonia,embarazo,habla_lengua_indig,diabetes,epoc,asma,inmusupr,hipertension,otra_com,cardiovascular,obesidad,renal_cronica,tabaquismo,otro_caso,migrante,uci,origen,sector,sexo,tipo_paciente,nacionalidad,resultado,defuncion,mes_def,año_def,mes_ingreso,año_ingreso
0,2020-09-14,063d1d,2020-04-15,2020-04-12,NaT,47,MÃ©xico,99,CIUDAD DE MÉXICO,AZCAPOTZALCO,CIUDAD DE MÉXICO,CIUDAD DE MÉXICO,NO APLICA,NO,NO,NO,NO,NO,NO,NO,NO,NO,NO,NO,NO,NO,SI,NO ESPECIFICADO,NO APLICA,FUERA DE USMER,IMSS,MUJER,AMBULATORIO,MEXICANA,Positivo SARS-CoV-2,False,NaN,NaN,4,2020
1,2020-09-14,0ee9be,2020-05-05,2020-05-04,NaT,38,MÃ©xico,99,CIUDAD DE MÉXICO,IZTAPALAPA,CIUDAD DE MÉXICO,CIUDAD DE MÉXICO,NO APLICA,NO,NO APLICA,NO,NO,NO,NO,NO,NO,NO,NO,NO,NO,NO,NO ESPECIFICADO,NO ESPECIFICADO,NO APLICA,FUERA DE USMER,IMSS,HOMBRE,AMBULATORIO,MEXICANA,Positivo SARS-CoV-2,False,NaN,NaN,5,2020


In [ ]:
# haciendo referencia a la columna como índice
data[data['nacionalidad'] == 'MEXICANA'].head(2)

,fecha_actualizacion,id_registro,fecha_ingreso,fecha_sintomas,fecha_def,edad,pais_nacionalidad,pais_origen,entidad_res,municipio_res,entidad_um,entidad_nac,intubado,neumonia,embarazo,habla_lengua_indig,diabetes,epoc,asma,inmusupr,hipertension,otra_com,cardiovascular,obesidad,renal_cronica,tabaquismo,otro_caso,migrante,uci,origen,sector,sexo,tipo_paciente,nacionalidad,resultado,defuncion,mes_def,año_def,mes_ingreso,año_ingreso
0,2020-09-03,097d14,2020-06-10,2020-06-10,NaT,27,MÃ©xico,99,BAJA CALIFORNIA,MEXICALI,BAJA CALIFORNIA,BAJA CALIFORNIA,NO APLICA,NO,NO APLICA,NO,NO,NO,NO,NO,NO,NO,NO,NO,NO,NO,SI,NO ESPECIFICADO,NO APLICA,FUERA DE USMER,ESTATAL,HOMBRE,AMBULATORIO,MEXICANA,Positivo SARS-CoV-2,False,NaN,NaN,6,2020
1,2020-09-03,1b7adb,2020-04-29,2020-04-15,2020-04-30,64,MÃ©xico,99,BAJA CALIFORNIA,TIJUANA,BAJA CALIFORNIA,GUERRERO,NO,SI,NO APLICA,NO,SI,NO,NO,NO,SI,NO,NO,NO,NO,NO,NO ESPECIFICADO,NO ESPECIFICADO,NO,FUERA DE USMER,IMSS,HOMBRE,HOSPITALIZADO,MEXICANA,Positivo SARS-CoV-2,True,4.0,2020.0,4,2020


In [ ]:
# seleccionando directamente la columna que queremos filtrar
data[data.nacionalidad == 'MEXICANA'].head(2)

,fecha_actualizacion,id_registro,fecha_ingreso,fecha_sintomas,fecha_def,edad,pais_nacionalidad,pais_origen,entidad_res,municipio_res,entidad_um,entidad_nac,intubado,neumonia,embarazo,habla_lengua_indig,diabetes,epoc,asma,inmusupr,hipertension,otra_com,cardiovascular,obesidad,renal_cronica,tabaquismo,otro_caso,migrante,uci,origen,sector,sexo,tipo_paciente,nacionalidad,resultado,defuncion,mes_def,año_def,mes_ingreso,año_ingreso
0,2020-09-03,097d14,2020-06-10,2020-06-10,NaT,27,MÃ©xico,99,BAJA CALIFORNIA,MEXICALI,BAJA CALIFORNIA,BAJA CALIFORNIA,NO APLICA,NO,NO APLICA,NO,NO,NO,NO,NO,NO,NO,NO,NO,NO,NO,SI,NO ESPECIFICADO,NO APLICA,FUERA DE USMER,ESTATAL,HOMBRE,AMBULATORIO,MEXICANA,Positivo SARS-CoV-2,False,NaN,NaN,6,2020
1,2020-09-03,1b7adb,2020-04-29,2020-04-15,2020-04-30,64,MÃ©xico,99,BAJA CALIFORNIA,TIJUANA,BAJA CALIFORNIA,GUERRERO,NO,SI,NO APLICA,NO,SI,NO,NO,NO,SI,NO,NO,NO,NO,NO,NO ESPECIFICADO,NO ESPECIFICADO,NO,FUERA DE USMER,IMSS,HOMBRE,HOSPITALIZADO,MEXICANA,Positivo SARS-CoV-2,True,4.0,2020.0,4,2020


**Filtrado múltiple**

In [ ]:
# usando query podemos poner en el string del query todas las condiciones que queramos
data.query('nacionalidad == "MEXICANA" & entidad_res == "CIUDAD DE MÉXICO"').head(2)

,fecha_actualizacion,id_registro,fecha_ingreso,fecha_sintomas,fecha_def,edad,pais_nacionalidad,pais_origen,entidad_res,municipio_res,entidad_um,entidad_nac,intubado,neumonia,embarazo,habla_lengua_indig,diabetes,epoc,asma,inmusupr,hipertension,otra_com,cardiovascular,obesidad,renal_cronica,tabaquismo,otro_caso,migrante,uci,origen,sector,sexo,tipo_paciente,nacionalidad,resultado,defuncion,mes_def,año_def,mes_ingreso,año_ingreso
22,2020-09-03,192ec3,2020-04-23,2020-04-23,2020-05-03,42,MÃ©xico,99,CIUDAD DE MÉXICO,AZCAPOTZALCO,CIUDAD DE MÉXICO,CIUDAD DE MÉXICO,NO,SI,NO APLICA,NO,NO,NO,NO,NO,NO,NO,NO,SI,NO,NO,NO ESPECIFICADO,NO ESPECIFICADO,NO,FUERA DE USMER,IMSS,HOMBRE,HOSPITALIZADO,MEXICANA,Positivo SARS-CoV-2,True,5.0,2020.0,4,2020
26,2020-09-03,1be6c6,2020-05-19,2020-05-18,NaT,37,MÃ©xico,99,CIUDAD DE MÉXICO,AZCAPOTZALCO,CIUDAD DE MÉXICO,MICHOACÁN DE OCAMPO,NO APLICA,NO,NO,NO,NO,NO,NO,NO,NO,NO,NO,NO,NO,NO,NO ESPECIFICADO,NO ESPECIFICADO,NO APLICA,FUERA DE USMER,IMSS,MUJER,AMBULATORIO,MEXICANA,Positivo SARS-CoV-2,False,NaN,NaN,5,2020


In [ ]:
# usando el filtrado tradicional debemos encerrar en paréntesis cada condición que queremos que cumpla nuestro filtro
data[(data['nacionalidad'] == "MEXICANA") & (data.entidad_res == "CIUDAD DE MÉXICO")].head(2)

,fecha_actualizacion,id_registro,fecha_ingreso,fecha_sintomas,fecha_def,edad,pais_nacionalidad,pais_origen,entidad_res,municipio_res,entidad_um,entidad_nac,intubado,neumonia,embarazo,habla_lengua_indig,diabetes,epoc,asma,inmusupr,hipertension,otra_com,cardiovascular,obesidad,renal_cronica,tabaquismo,otro_caso,migrante,uci,origen,sector,sexo,tipo_paciente,nacionalidad,resultado,defuncion,mes_def,año_def,mes_ingreso,año_ingreso
22,2020-09-03,192ec3,2020-04-23,2020-04-23,2020-05-03,42,MÃ©xico,99,CIUDAD DE MÉXICO,AZCAPOTZALCO,CIUDAD DE MÉXICO,CIUDAD DE MÉXICO,NO,SI,NO APLICA,NO,NO,NO,NO,NO,NO,NO,NO,SI,NO,NO,NO ESPECIFICADO,NO ESPECIFICADO,NO,FUERA DE USMER,IMSS,HOMBRE,HOSPITALIZADO,MEXICANA,Positivo SARS-CoV-2,True,5.0,2020.0,4,2020
26,2020-09-03,1be6c6,2020-05-19,2020-05-18,NaT,37,MÃ©xico,99,CIUDAD DE MÉXICO,AZCAPOTZALCO,CIUDAD DE MÉXICO,MICHOACÁN DE OCAMPO,NO APLICA,NO,NO,NO,NO,NO,NO,NO,NO,NO,NO,NO,NO,NO,NO ESPECIFICADO,NO ESPECIFICADO,NO APLICA,FUERA DE USMER,IMSS,MUJER,AMBULATORIO,MEXICANA,Positivo SARS-CoV-2,False,NaN,NaN,5,2020


###Resúmenes

In [ ]:
# una columna, una métrica
data.groupby('sexo').edad.mean()

sexo
HOMBRE    42.812564
MUJER     41.735627
Name: edad, dtype: float64

In [ ]:
# una columna, varias métricas
data.groupby('sexo').agg({'edad': ['min','mean','max']})

edad                
        min       mean  max
sexo                       
HOMBRE    0  42.812564  120
MUJER     0  41.735627  120

In [ ]:
# varias columnas, una métrica
data.groupby('sexo').agg({'edad': 'mean', 'mes_ingreso': 'mean', 'mes_def': 'mean'})

,edad,mes_ingreso,mes_def
sexo,,,
HOMBRE,42.812564,6.548249,6.371355
MUJER,41.735627,6.611875,6.431488


In [ ]:
# varias columnas, varias métricas
data.groupby('sexo').agg({'edad': ['min','mean','max'], 'mes_ingreso': ['min','mean','max'], 'mes_def': ['min','mean','max']})

edad                 mes_ingreso               mes_def               
        min       mean  max         min      mean max     min      mean  max
sexo                                                                        
HOMBRE    0  42.812564  120           1  6.548249   9     1.0  6.371355  9.0
MUJER     0  41.735627  120           1  6.611875   9     1.0  6.431488  9.0

###Descripción por Tipo

In [ ]:
# Filtrar por tipos generales (categóricas y numéricas)
numerical = data.select_dtypes(include='number')
categorical = data.select_dtypes(include=['object', 'bool', 'category', 'datetime64[ns]'])

In [ ]:
numerical.head()

,edad,mes_def,año_def,mes_ingreso,año_ingreso
0,27,NaN,NaN,6,2020
1,64,4.0,2020.0,4,2020
2,32,NaN,NaN,5,2020
3,54,NaN,NaN,6,2020
4,44,NaN,NaN,4,2020


In [ ]:
categorical.head()

,fecha_actualizacion,id_registro,fecha_ingreso,fecha_sintomas,fecha_def,pais_nacionalidad,pais_origen,entidad_res,municipio_res,entidad_um,entidad_nac,intubado,neumonia,embarazo,habla_lengua_indig,diabetes,epoc,asma,inmusupr,hipertension,otra_com,cardiovascular,obesidad,renal_cronica,tabaquismo,otro_caso,migrante,uci,origen,sector,sexo,tipo_paciente,nacionalidad,resultado,defuncion
0,2020-09-03,097d14,2020-06-10,2020-06-10,NaT,MÃ©xico,99,BAJA CALIFORNIA,MEXICALI,BAJA CALIFORNIA,BAJA CALIFORNIA,NO APLICA,NO,NO APLICA,NO,NO,NO,NO,NO,NO,NO,NO,NO,NO,NO,SI,NO ESPECIFICADO,NO APLICA,FUERA DE USMER,ESTATAL,HOMBRE,AMBULATORIO,MEXICANA,Positivo SARS-CoV-2,False
1,2020-09-03,1b7adb,2020-04-29,2020-04-15,2020-04-30,MÃ©xico,99,BAJA CALIFORNIA,TIJUANA,BAJA CALIFORNIA,GUERRERO,NO,SI,NO APLICA,NO,SI,NO,NO,NO,SI,NO,NO,NO,NO,NO,NO ESPECIFICADO,NO ESPECIFICADO,NO,FUERA DE USMER,IMSS,HOMBRE,HOSPITALIZADO,MEXICANA,Positivo SARS-CoV-2,True
2,2020-09-03,11daaa,2020-05-26,2020-05-26,NaT,MÃ©xico,99,TABASCO,TACOTALPA,TABASCO,TABASCO,NO APLICA,NO,NO,NO ESPECIFICADO,NO,NO,NO,NO,NO,NO,NO,NO,NO,NO,SI,NO ESPECIFICADO,NO APLICA,FUERA DE USMER,ESTATAL,MUJER,AMBULATORIO,MEXICANA,Positivo SARS-CoV-2,False
3,2020-09-03,17c9eb,2020-06-05,2020-05-31,NaT,MÃ©xico,99,TABASCO,NACAJUCA,TABASCO,TABASCO,NO APLICA,NO,NO,NO,SI,NO,NO,NO,SI,NO,NO,NO,NO,NO,SI,NO ESPECIFICADO,NO APLICA,FUERA DE USMER,ESTATAL,MUJER,AMBULATORIO,MEXICANA,Positivo SARS-CoV-2,False
4,2020-09-03,14f12c,2020-04-02,2020-04-01,NaT,MÃ©xico,99,MORELOS,CUERNAVACA,MORELOS,BAJA CALIFORNIA SUR,NO APLICA,NO,NO,NO,NO,NO,NO,NO,NO,SI,NO,NO,NO,NO,SI,NO ESPECIFICADO,NO APLICA,FUERA DE USMER,IMSS,MUJER,AMBULATORIO,MEXICANA,Positivo SARS-CoV-2,False


In [ ]:
# para conocer todos los tipo de datos
?data.select_dtypes

####Categóricas

In [ ]:
# Cardinalidad - Valores únicos por columna
for col in categorical.columns:
  print('Valores únicos en "{}": {}'.format(col,categorical[col].nunique()))

Valores únicos en "fecha_actualizacion": 1
Valores únicos en "id_registro": 1393033
Valores únicos en "fecha_ingreso": 247
Valores únicos en "fecha_sintomas": 247
Valores únicos en "fecha_def": 192
Valores únicos en "pais_nacionalidad": 119
Valores únicos en "pais_origen": 78
Valores únicos en "entidad_res": 32
Valores únicos en "municipio_res": 2155
Valores únicos en "entidad_um": 32
Valores únicos en "entidad_nac": 33
Valores únicos en "intubado": 4
Valores únicos en "neumonia": 3
Valores únicos en "embarazo": 4
Valores únicos en "habla_lengua_indig": 3
Valores únicos en "diabetes": 3
Valores únicos en "epoc": 3
Valores únicos en "asma": 3
Valores únicos en "inmusupr": 3
Valores únicos en "hipertension": 3
Valores únicos en "otra_com": 3
Valores únicos en "cardiovascular": 3
Valores únicos en "obesidad": 3
Valores únicos en "renal_cronica": 3
Valores únicos en "tabaquismo": 3
Valores únicos en "otro_caso": 3
Valores únicos en "migrante": 3
Valores únicos en "uci": 4
Valores únicos en

In [ ]:
# Unicidad - proporción de valores únicos por columna
total = data.shape[0]
for col in categorical.columns:
  print('Porcentaje de valores únicos en "{}": {}%'.format(col,round(categorical[col].nunique()/total*100,2)))

Porcentaje de valores únicos en "fecha_actualizacion": 0.0%
Porcentaje de valores únicos en "id_registro": 100.0%
Porcentaje de valores únicos en "fecha_ingreso": 0.02%
Porcentaje de valores únicos en "fecha_sintomas": 0.02%
Porcentaje de valores únicos en "fecha_def": 0.01%
Porcentaje de valores únicos en "pais_nacionalidad": 0.01%
Porcentaje de valores únicos en "pais_origen": 0.01%
Porcentaje de valores únicos en "entidad_res": 0.0%
Porcentaje de valores únicos en "municipio_res": 0.15%
Porcentaje de valores únicos en "entidad_um": 0.0%
Porcentaje de valores únicos en "entidad_nac": 0.0%
Porcentaje de valores únicos en "intubado": 0.0%
Porcentaje de valores únicos en "neumonia": 0.0%
Porcentaje de valores únicos en "embarazo": 0.0%
Porcentaje de valores únicos en "habla_lengua_indig": 0.0%
Porcentaje de valores únicos en "diabetes": 0.0%
Porcentaje de valores únicos en "epoc": 0.0%
Porcentaje de valores únicos en "asma": 0.0%
Porcentaje de valores únicos en "inmusupr": 0.0%
Porcenta

In [ ]:
# Moda - valor que más se repite en cada columna excepto los indicadores únicos
for col in categorical.drop(columns='id_registro').columns:
  print('Valor más popular en "{}": {}'.format(col, statistics.mode(categorical[col])))

Valor más popular en "fecha_actualizacion": 2020-09-03 00:00:00
Valor más popular en "fecha_ingreso": 2020-07-20 00:00:00
Valor más popular en "fecha_sintomas": 2020-07-20 00:00:00
Valor más popular en "fecha_def": NaT
Valor más popular en "pais_nacionalidad": MÃ©xico
Valor más popular en "pais_origen": 99
Valor más popular en "entidad_res": CIUDAD DE MÉXICO
Valor más popular en "municipio_res": IZTAPALAPA
Valor más popular en "entidad_um": CIUDAD DE MÉXICO
Valor más popular en "entidad_nac": CIUDAD DE MÉXICO
Valor más popular en "intubado": NO APLICA
Valor más popular en "neumonia": NO 
Valor más popular en "embarazo": NO 
Valor más popular en "habla_lengua_indig": NO 
Valor más popular en "diabetes": NO 
Valor más popular en "epoc": NO 
Valor más popular en "asma": NO 
Valor más popular en "inmusupr": NO 
Valor más popular en "hipertension": NO 
Valor más popular en "otra_com": NO 
Valor más popular en "cardiovascular": NO 
Valor más popular en "obesidad": NO 
Valor más popular en "r

####Numéricas

In [ ]:
numerical.describe()

,edad,mes_def,año_def,mes_ingreso,año_ingreso
count,1.393033e+06,84677.000000,84677.0,1.393033e+06,1393033.0
mean,4.226682e+01,6.393200,2020.0,6.580492e+00,2020.0
std,1.678013e+01,1.218526,0.0,1.288985e+00,0.0
min,0.000000e+00,1.000000,2020.0,1.000000e+00,2020.0
25%,3.000000e+01,5.000000,2020.0,6.000000e+00,2020.0
50%,4.100000e+01,6.000000,2020.0,7.000000e+00,2020.0
75%,5.300000e+01,7.000000,2020.0,8.000000e+00,2020.0
max,1.200000e+02,9.000000,2020.0,9.000000e+00,2020.0


##Análisis Exploratorio

En esta sección responderemos algunas preguntas sobre el set de datos usando las herramientas que hemos visto hasta ahora

In [ ]:
# ¿Cuántas pruebas por origen se realizan?
data.groupby('origen').id_registro.count()

origen
FUERA DE USMER    928460
USMER             464573
Name: id_registro, dtype: int64

In [ ]:
data.origen.value_counts()

FUERA DE USMER    928460
USMER             464573
Name: origen, dtype: int64

In [ ]:
# ¿Cuántas pruebas por sector se realizan?
data.sector.value_counts()

SSA                863605
IMSS               359287
ISSSTE              52959
PRIVADA             52236
ESTATAL             25729
PEMEX               12225
IMSS-BIENESTAR       9690
SEMAR                7387
SEDENA               6890
MUNICIPAL            1128
UNIVERSITARIO         897
DIF                   786
CRUZ ROJA             206
NO ESPECIFICADO         8
Name: sector, dtype: int64

In [ ]:
# ¿Cuántos pacientes por sexo tenemos?
data.sexo.value_counts()

MUJER     705922
HOMBRE    687111
Name: sexo, dtype: int64

In [ ]:
# ¿Cuál es la proporción de los pacientes por sexo?
data.sexo.value_counts(normalize=True)

MUJER     0.506752
HOMBRE    0.493248
Name: sexo, dtype: float64

In [ ]:
# ¿Cuál es la proporción de resultados de las pruebas?
data.resultado.value_counts(normalize=True)

No positivo SARS-CoV-2    0.496987
Positivo SARS-CoV-2       0.442842
Resultado pendiente       0.060171
Name: resultado, dtype: float64

In [ ]:
# ¿Cuántas pruebas positivas tenemos en el set?
data[data.resultado == 'Positivo SARS-CoV-2'].id_registro.count()

616894

In [ ]:
# ¿Cuántos casos positivos tenemos por mes?
data[data.resultado == 'Positivo SARS-CoV-2'].mes_ingreso.value_counts().sort_index()

1         2
2         8
3      2530
4     26554
5     86466
6    152756
7    198622
8    148077
9      1879
Name: mes_ingreso, dtype: int64

In [ ]:
# ¿Cuál es el resultado de las pruebas por sexo?
data.groupby(['sexo','resultado'])['id_registro'].count()

sexo    resultado             
HOMBRE  No positivo SARS-CoV-2    322529
        Positivo SARS-CoV-2       323257
        Resultado pendiente        41325
MUJER   No positivo SARS-CoV-2    369790
        Positivo SARS-CoV-2       293637
        Resultado pendiente        42495
Name: id_registro, dtype: int64

In [ ]:
data.groupby('sexo').resultado.value_counts()

sexo    resultado             
HOMBRE  Positivo SARS-CoV-2       323257
        No positivo SARS-CoV-2    322529
        Resultado pendiente        41325
MUJER   No positivo SARS-CoV-2    369790
        Positivo SARS-CoV-2       293637
        Resultado pendiente        42495
Name: resultado, dtype: int64

In [ ]:
# ¿Qué proporción de resultados de pruebas tenemos por sexo?
data.groupby('sexo').resultado.value_counts(normalize=True)

sexo    resultado             
HOMBRE  Positivo SARS-CoV-2       0.470458
        No positivo SARS-CoV-2    0.469399
        Resultado pendiente       0.060143
MUJER   No positivo SARS-CoV-2    0.523840
        Positivo SARS-CoV-2       0.415962
        Resultado pendiente       0.060198
Name: resultado, dtype: float64

In [ ]:
# ¿Cuál es la edad de los pacientes por sexo?
data.groupby('sexo').edad.describe()

,count,mean,std,min,25%,50%,75%,max
sexo,,,,,,,,
HOMBRE,687111.0,42.812564,17.091827,0.0,30.0,41.0,54.0,120.0
MUJER,705922.0,41.735627,16.453715,0.0,30.0,40.0,52.0,120.0


In [ ]:
def q25(x):
  return x.quantile(0.25)

def q75(x):
  return x.quantile(0.75)

data.groupby('sexo').agg({'edad': ['min', q25, 'median', q75, 'max']})

edad                    
        min q25 median q75  max
sexo                           
HOMBRE    0  30     41  54  120
MUJER     0  30     40  52  120

In [ ]:
# ¿Cuál es la edad de los pacientes dependiendo de si son intubados o no?
data.groupby('intubado').edad.describe()

,count,mean,std,min,25%,50%,75%,max
intubado,,,,,,,,
NO,219583.0,53.630053,19.435595,0.0,42.0,55.0,67.0,120.0
NO APLICA,1144649.0,39.748604,15.020323,0.0,29.0,38.0,49.0,120.0
NO ESPECIFICADO,167.0,53.005988,18.042619,0.0,41.5,54.0,65.5,92.0
SI,28634.0,55.730286,18.850803,0.0,47.0,58.0,68.0,108.0


In [ ]:
data.groupby('intubado').agg({'edad': ['min', q25, 'median', q75, 'max']})

edad                        
                 min   q25 median   q75  max
intubado                                    
NO                 0  42.0     55  67.0  120
NO APLICA          0  29.0     38  49.0  120
NO ESPECIFICADO    0  41.5     54  65.5   92
SI                 0  47.0     58  68.0  108

In [ ]:
# ¿Cuál es la edad máxima y cuál es la mínima de los pacientes fallecidos?
edad_def_max = data[data.defuncion].edad.max()
data[data.defuncion & (data.edad == edad_def_max)]

,fecha_actualizacion,id_registro,fecha_ingreso,fecha_sintomas,fecha_def,edad,pais_nacionalidad,pais_origen,entidad_res,municipio_res,entidad_um,entidad_nac,intubado,neumonia,embarazo,habla_lengua_indig,diabetes,epoc,asma,inmusupr,hipertension,otra_com,cardiovascular,obesidad,renal_cronica,tabaquismo,otro_caso,migrante,uci,origen,sector,sexo,tipo_paciente,nacionalidad,resultado,defuncion,mes_def,año_def,mes_ingreso,año_ingreso
551494,2020-09-03,1b7b20,2020-05-21,2020-05-20,2020-05-26,109,MÃ©xico,99,MICHOACÁN DE OCAMPO,APATZINGÁN,MICHOACÁN DE OCAMPO,MICHOACÁN DE OCAMPO,NO,SI,NO APLICA,NO,NO,NO,NO,SI,NO,NO,NO,NO,SI,NO,NO,NO ESPECIFICADO,NO,FUERA DE USMER,ISSSTE,HOMBRE,HOSPITALIZADO,MEXICANA,No positivo SARS-CoV-2,True,5.0,2020.0,5,2020


In [54]:
edad_def_min = data[data.defuncion].edad.min()
data[data.defuncion & (data.edad == edad_def_min)]

,fecha_actualizacion,id_registro,fecha_ingreso,fecha_sintomas,fecha_def,edad,pais_nacionalidad,pais_origen,entidad_res,municipio_res,entidad_um,entidad_nac,intubado,neumonia,embarazo,habla_lengua_indig,diabetes,epoc,asma,inmusupr,hipertension,otra_com,cardiovascular,obesidad,renal_cronica,tabaquismo,otro_caso,migrante,uci,origen,sector,sexo,tipo_paciente,nacionalidad,resultado,defuncion,mes_def,año_def,mes_ingreso,año_ingreso
1287,2020-09-14,083c34,2020-05-25,2020-05-25,2020-06-01,0,MÃ©xico,99,MÉXICO,CHIMALHUACÁN,MÉXICO,MÉXICO,SI,SI,NO APLICA,NO,NO,NO,NO,NO,NO,NO,NO,NO,NO,NO,SI,NO ESPECIFICADO,SI,USMER,IMSS,HOMBRE,HOSPITALIZADO,MEXICANA,Positivo SARS-CoV-2,True,6.0,2020.0,5,2020
28987,2020-09-14,1cc0ea,2020-04-24,2020-04-21,2020-04-29,0,MÃ©xico,99,MÉXICO,ECATEPEC DE MORELOS,MÉXICO,MÉXICO,NO,SI,NO,NO,SI,NO,NO,NO,SI,NO,NO,NO,NO,NO,NO ESPECIFICADO,NO ESPECIFICADO,NO,FUERA DE USMER,SSA,MUJER,HOSPITALIZADO,MEXICANA,Positivo SARS-CoV-2,True,4.0,2020.0,4,2020
31662,2020-09-14,014b5d,2020-05-18,2020-05-17,2020-05-18,0,MÃ©xico,99,BAJA CALIFORNIA,TIJUANA,BAJA CALIFORNIA,BAJA CALIFORNIA,SI,SI,NO APLICA,NO,NO,NO,NO,NO,NO,NO,NO,NO,NO,NO,NO,NO ESPECIFICADO,NO,FUERA DE USMER,ISSSTE,HOMBRE,HOSPITALIZADO,MEXICANA,Positivo SARS-CoV-2,True,5.0,2020.0,5,2020
37142,2020-09-14,02f8e2,2020-05-08,2020-05-01,2020-05-10,0,MÃ©xico,99,BAJA CALIFORNIA,TIJUANA,BAJA CALIFORNIA,BAJA CALIFORNIA,SI,SI,NO APLICA,NO,NO,NO,NO,NO,NO,NO,NO,NO,NO,NO,NO,NO ESPECIFICADO,NO,USMER,SSA,HOMBRE,HOSPITALIZADO,MEXICANA,Positivo SARS-CoV-2,True,5.0,2020.0,5,2020
38166,2020-09-14,1c724e,2020-04-19,2020-04-14,2020-04-21,0,MÃ©xico,99,MÉXICO,CHIMALHUACÁN,MÉXICO,MÉXICO,NO,SI,NO APLICA,NO,NO,NO,NO,NO,NO,NO,NO,SI,NO,NO,SI,NO ESPECIFICADO,NO,FUERA DE USMER,SSA,HOMBRE,HOSPITALIZADO,MEXICANA,Positivo SARS-CoV-2,True,4.0,2020.0,4,2020
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1454035,2020-09-14,082242,2020-08-07,2020-08-06,2020-08-10,0,MÃ©xico,99,ZACATECAS,ZACATECAS,ZACATECAS,ZACATECAS,NO,NO,NO APLICA,NO,NO,NO,NO,NO,NO,NO,NO,NO,NO,NO,NO ESPECIFICADO,NO ESPECIFICADO,SI,USMER,IMSS,HOMBRE,HOSPITALIZADO,MEXICANA,No positivo SARS-CoV-2,True,8.0,2020.0,8,2020
1478501,2020-09-14,1e94e4,2020-08-14,2020-08-06,2020-08-20,0,MÃ©xico,99,SAN LUIS POTOSÍ,SAN LUIS POTOSÍ,SAN LUIS POTOSÍ,SAN LUIS POTOSÍ,SI,SI,NO APLICA,NO,NO,NO,NO,NO,NO,NO,NO,NO,NO,NO,SI,NO ESPECIFICADO,SI,FUERA DE USMER,PRIVADA,HOMBRE,HOSPITALIZADO,MEXICANA,Resultado pendiente,True,8.0,2020.0,8,2020
1486980,2020-09-14,00f39c,2020-07-06,2020-07-03,2020-07-08,0,MÃ©xico,99,JALISCO,ZAPOTILTIC,JALISCO,JALISCO,NO,SI,NO APLICA,NO,NO,NO,NO,NO,NO,NO,NO,NO,NO,NO,NO,NO ESPECIFICADO,SI,USMER,SSA,HOMBRE,HOSPITALIZADO,MEXICANA,Resultado pendiente,True,7.0,2020.0,7,2020
1502749,2020-09-14,07cf0b,2020-08-26,2020-08-25,2020-08-27,0,MÃ©xico,99,CHIHUAHUA,JUÁREZ,CHIHUAHUA,COAHUILA DE ZARAGOZA,NO,SI,NO,NO,NO,NO,NO,NO,NO,SE IGNORA,NO,NO,NO,NO,NO ESPECIFICADO,NO ESPECIFICADO,NO,USMER,SSA,MUJER,HOSPITALIZADO,MEXICANA,Resultado pendiente,True,8.0,2020.0,8,2020


In [ ]:
# ¿Cuál es la edad de los pacientes que fallecen?
data.groupby('defuncion').edad.describe()

,count,mean,std,min,25%,50%,75%,max
defuncion,,,,,,,,
False,1308356.0,41.001587,16.077933,0.0,30.0,40.0,51.0,120.0
True,84677.0,61.816172,15.204152,0.0,53.0,63.0,72.0,109.0


In [ ]:
# ¿Cuántas defunciones tenemos por mes?
data[data.defuncion].mes_def.value_counts().sort_index()

1.0        6
2.0       12
3.0      232
4.0     5277
5.0    16018
6.0    21318
7.0    23471
8.0    17859
9.0      484
Name: mes_def, dtype: int64

In [ ]:
# ¿Cuál es la proporción de diábeticos para las defunciones?
data.groupby('diabetes').defuncion.value_counts(normalize=True)

diabetes   defuncion
NO         False        0.957313
           True         0.042687
SE IGNORA  False        0.873791
           True         0.126209
SI         False        0.812020
           True         0.187980
Name: defuncion, dtype: float64

In [ ]:
# ¿Cuál es la proporción de asmáticos para las defunciones?
data.groupby('asma').defuncion.value_counts(normalize=True)

asma       defuncion
NO         False        0.938871
           True         0.061129
SE IGNORA  False        0.865200
           True         0.134800
SI         False        0.957209
           True         0.042791
Name: defuncion, dtype: float64

# TAREA 3



**¿Tienen los pacientes con hipertensión un riesgo más alto de defunción?**





In [28]:
# Agrupar por enfermedad y luego hacer una normalización por estados
data.groupby('hipertension').defuncion.value_counts(normalize=True)

hipertension  defuncion
NO            False        0.960773
              True         0.039227
SE IGNORA     False        0.872214
              True         0.127786
SI            False        0.834681
              True         0.165319
Name: defuncion, dtype: float64

**¿Cuántos casos confirmados se tienen por Estado?**

In [31]:
# filtrar los resultados positivos con entidad de residencia, aplicar conteo.
data[data.resultado == 'Positivo SARS-CoV-2'].entidad_res.value_counts().sort_index()

AGUASCALIENTES                       6441
BAJA CALIFORNIA                     18346
BAJA CALIFORNIA SUR                  9033
CAMPECHE                             5816
CHIAPAS                              6393
CHIHUAHUA                            9013
CIUDAD DE MÉXICO                   111897
COAHUILA DE ZARAGOZA                24267
COLIMA                               4231
DURANGO                              7821
GUANAJUATO                          36973
GUERRERO                            16600
HIDALGO                             11514
JALISCO                             23346
MICHOACÁN DE OCAMPO                 17844
MORELOS                              5552
MÉXICO                              74214
NAYARIT                              5502
NUEVO LEÓN                          34191
OAXACA                              14925
PUEBLA                              29259
QUERÉTARO                            7722
QUINTANA ROO                        11124
SAN LUIS POTOSÍ                   

**¿Cuántas defunciones se tienen por Estado?**

In [32]:
# filtrar las defunciones globales y agrupar por entidad de residencia, aplicar sumatoria.
data[data.defuncion].entidad_res.value_counts().sort_index()

AGUASCALIENTES                       605
BAJA CALIFORNIA                     4091
BAJA CALIFORNIA SUR                  541
CAMPECHE                             967
CHIAPAS                             1221
CHIHUAHUA                           2045
CIUDAD DE MÉXICO                   12086
COAHUILA DE ZARAGOZA                2225
COLIMA                               606
DURANGO                              721
GUANAJUATO                          3230
GUERRERO                            2307
HIDALGO                             2111
JALISCO                             3949
MICHOACÁN DE OCAMPO                 1948
MORELOS                             1257
MÉXICO                             13724
NAYARIT                              827
NUEVO LEÓN                          3567
OAXACA                              1630
PUEBLA                              4647
QUERÉTARO                           1019
QUINTANA ROO                        1937
SAN LUIS POTOSÍ                     1793
SINALOA         

**¿Cuántos fallecimientos han ocurrido en el Estado con mayor número de casos confirmados?**

In [63]:
# seleccionar el estado del de mayor número de casos confirmados
max_confirm_state = data[data.resultado == 'Positivo SARS-CoV-2'].entidad_res.value_counts().idxmax()
# filtramos por defuncion y entidad de mayor número de casos confirmados
data[data.defuncion & (data.entidad_res == max_confirm_state)].entidad_res.value_counts()

CIUDAD DE MÉXICO    12086
Name: entidad_res, dtype: int64

 **¿Cuántos fallecimientos han ocurrido en los pacientes Ambulatorios?**

In [71]:
data[data.defuncion & (data.tipo_paciente == 'AMBULATORIO')].tipo_paciente.value_counts()

AMBULATORIO    9651
Name: tipo_paciente, dtype: int64